# Finite Markov Decision Processes

### Intro.

MDP는 순차적 의사결정(sequential decision making) 모델로써, 행동(action)은 즉각적인 보상(rewards) 뿐만 아니라 이후 순차적인 상태(states)들의 보상(future rewards)을 고려하여 결정된다. 그러므로 MDP에서는 지연보상(delayed rewards)과 즉각적 보상(immediate rewards) 사이에서의 적절한 균형을 염두할 필요가 있다. 또한 각 action에 대한 장기적인 가치를 정확히 매기기 위해, Multi-armed bandit 문제에서 $q_*(a)$를 추정했던 것에 달리, MDP에서는 state s에 대한 action a의 value인 $q_*(s, a)$와 각 state에 대한 value인 $v_*(s)$(state-dependent quantities)를 추정한다.

MDP는 수학적으로 정확히 강화학습 문제를 서술하기 위한 이상적인 형태이다. 이 챕터에서는 문제들을 수학적인 구조로 풀어낼때 중요한 역할을 하는 요소들인 returns, value functions, Bellman equations 같은 개념들을 소개한다. 더불어 finite MDP로 정형화할 수 있는 몇 가지 응용예시들을 살펴보도록 한다.

### 3.1 The Agent-Environment Interface

<img src="images/fig3.1.png" alt="The agent-environment interaction in MDP"/>

앞서(챕터1) 강화학습을 목표지향적 상호작용 학습(goal-directed learning from interation)이라 표현하였다. MDP를 이용하면 이러한 문제를 단순한 형태로 표현할 수 있게 된다. 여기서 '상호작용'에 대해 좀 더 자세히 살펴보도록 하자. 위 그림 3.1처럼 상호작용은 agent와 environment(환경)사이에서 지속적으로 이루어진다. 이때 *agent*란 의사결정과 학습을 하는 주체를 뜻하며, *environment*란 agent가 상호작용하는 agent 외의 모든 것을 의미한다. 

Agent와 environment의 상호작용(interaction)은 순차적(sequential)이고 이산적인(discrete) time step, $t=1,2,3,\dots$ 에 걸쳐서 발생한다. 특정 time step $t$에서 agent는 environment에 대한 상태, $S_t \in \mathcal{S}$ 를 받아 이를 기반으로 action, $A_t \in \mathcal{A}$ 을 선택하게 된다. 그리고 한 time step 뒤, action에 대한 결과로써 agent는 수치적인(numerical) 보상, $R_{t+1} \in \mathcal{R} \subset \mathbb{R}$ 과 다음 상태 $S_{t+1}$를 받는다. 이는 곧 아래와 같은 trajectory로 표현할 수 있다.
$$S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, R_3, \dots$$

*Finite* MDP에서 states, actions, rewards ($\mathcal{S}, \mathcal{A}, \mathcal{R}$)에 대한 집합은 유한한 갯수의 원소로 구성되어 있다. 이 경우, 어떤 시점 $t$ 에서 앞선 state $s \in \mathcal{S}$ 와 action $a \in \mathcal{A}$ 에 대해 random variables $S_t$와 $R_t$이 어떤 특정한 값 $s' \in \mathcal{S}, r \in \mathcal{R}$ 을 가질 확률은 다음과 같이 표현된다.
$$
p(s', r \: | \: s, a) \doteq Pr\{S_t = s', R_t = r \: | \: S_{t-1} = s, A_{t-1} = a \},\\
\text{for all } \: s', s \in \mathcal{S}, \text{ and } a \in \mathcal{A}(s).
$$

함수 $p: \mathcal{S} \: \times \: \mathcal{R} \: \times \: \mathcal{S} \: \times \: \mathcal{A} \rightarrow [0, 1]$ 는 4개 인자에 대한 ordinary deterministic function이며, 다음을 만족하는 확률분포를 명시한다.
$$
\sum_{s' \in \mathcal{S}} \sum_{r \in \mathcal{R}} p(s', r \:| \: s, a) = 1, \text{ for all } \: s \in \mathcal{S}, a \in \mathcal{A}(s).
$$

책에서는 이 4개 인자에 대한 $p$ 함수가 주로 사용된다. 더불어 $p$ 함수를 이용해 유도할 수 있는 다른 몇 가지 notation들에 대해서도 추가적으로 살펴보도록 하자.

* *state-transition probabilities: *
$$
p(s' \: | \: s, a) \doteq Pr\{S_t \: | \: S_{t-1}=s, A_{t-1} = a \} = \sum_{r \in \mathcal{R}} p (s', r \: | \: s, a).
$$

* *expected rewards: *
$$
r(s,a) \doteq \mathbb{E}[ R_t \: | \: S_{t-1} = s, A_{t-1} = a ] = \sum_{r \in \mathcal{R}} r \sum_{s' \in \mathcal{S}} p (s', r \: | \: s, a).
$$

* *expected rewards for state-action-next-state triples: *
$$
r(s,a,s') \doteq \mathbb{E}[ R_t \: | \: S_{t-1} = s, A_{t-1} = a, S_t = s' ] = \sum_{r \in \mathcal{R}} r \frac{p (s', r \: | \: s, a)}{p(s' \: | \: s, a)}.
$$

MDP framework는 목표지향적인 상호작용 학습(goal-directed learning from interation) 문제를 agent와 environment 사이에 3가지의 신호가 오가는 형태로 추상화한다. 바로 agent의 의사결정을 나타내는 신호(action), 그 의사결정의 근거가 되는 신호(state), 그리고 agent의 목표를 정의하는 신호(reward)다. 이 framework가 모든 decision-learning 문제를 충분히 잘 표현할 수 있다고 할 수는 없지만, 이 방식의 유용함과 다양한 적용가능성은 이미 널리 입증되어 있다. (This framework may not be sufficient to represent all decision-learning problems usefully, but it has proved to be widely useful and applicable.)

**Example 3.3** Recycling Robot

빈 깡통을 수거하는 로봇이 있다고 해보자. 로봇은 충전식 배터리로 구동되며, 깡통을 감지하면 팔을 이용하여 그것을 집은 뒤 쓰레기통에 집어넣는다. 이때 가장 고차원의 의사결정은 배터리의 잔량에 근거한 강화학습을 통해 이루어진다. 간단한 예시를 들어보자. 배터리는 다음의 두 가지 상태로 구분된다: $\mathcal{S} = \{high, low \}$. 각 상태에서 로봇은 깡통을 탐색(search)하거나, 누군가가 깡통을 집어주길 기다리거나(wait), 충전소로 돌아갈 수 있다(recharge). 상태에 따른 action set은 다음과 같다: $\mathcal{A}(high) = \{search, wait\}$ and $\mathcal{A}(low) = \{search, wait, recharge\}$. 보상은 대부분의 경우 0이지만 깡통을 수거하면 양수($r_\text{search}$)가, 배터리가 바닥나면 음수(-3)가 된다. 그리고 탐색과 대기에 대한 보상은 항상 $r_\text{search} > r_\text{wait}$가 되도록 설정한다.

다음은 각 상태간에 천이(transition)가 일어날 확률에 대해 이야기해보자. $high$ 상태에서 탐색을 할때, 배터리 잔량이 여전히 $high$로 유지될 가능성을 $\alpha$, 그렇지 않을 가능성을 $1 - \alpha$라 하겠다. 또한 상태가 $low$일때 탐색 후에도 여전히 배터리 잔량이 $low$로 유지될 가능성은 $\beta$, 배터리가 소진되어 사용자의 손에 의해 충전될(rescued) 가능성은 $1 - \beta$라고 가정한다.

마지막으로 충전소로 돌아가는 길에는 깡통을 수거할 수 없다고 하자. 앞서 서술한 내용을 finite MDP로 표현해보면 다음과 같다.
<img src="images/example3.3.png" alt="Recycling Robot"/>
Finite MDP는 좌측과 같이 테이블로 표현되거나 혹은 우측처럼 *state node*와 *action node*로 구성된 *transition graph*로도 표현될 수 있다. 여기서 같은 state-action에 대한 transition들의 모든 확률의 총합은 항상 1임을 유의하자.

### 3.2 Goals and Rewards

강화학습에서 agent의 목표는 환경에서 agent로 전달되는 *보상(reward)*라는 특수한 신호를 통해 정형화(formalized)된다. 각 time step에서 보상은 어떤 임의의 실수로 표현되며, agent의 목표는 이 reward의 누적합에 대한 기대값을 최대화하는 것에 있다(*reward hypothesis*). Reward signal을 통해 목표를 설정하는 방식은 언뜻 보면 제약이 많은 것처럼 느껴질 수 있으나, 실전에서 이 방식은 유연하게 널리 적용될 수 있음이 증명되어있다. 

몇 가지 예를 살펴보겠다. 
* 미로를 탈출하는 로봇을 학습시킨다고 해보자. 탈출하기까지 매 step마다 -1의 보상을 주게 된다면 이는 agent가 탈츨을 최대한 빠르게 할 수 있도록 부추기는 효과를 일으킬 것이다. 
* 재활용 깡통을 수거하는 로봇을 학습시킨다고 해보자. 대부분의 경우 0의 보상을, 깡통을 성공적으로 수거했을때는 양(positive)의 보상을, 무언가의 부딪치거나 행인에게 폐를 끼쳤을때는 음(negative)의 보상을 설정할 수 있을 것이다.
* 체스 게임을 학습하는 agent가 있다고 하면, 승리에는 +1을, 패배에는 -1을, 비긴 경우에는 0을 보상으로 설정할 수 있다.

주의할 점은 reward signal을 통해 목적 달성하는 *방법*을 구체적으로 의도하는 것은 부적절하다는 것이다. 가령, 체승게임에서 최종 승리에 대한 보상 외에 부차적인 목표를 설정하게 되면 (e.g. 최대한 상대방의 패를 많이 잡기), agent는 부차적인 목표를 성취하기 위해 본래 목표에는 도달하지 못하게 될 수도 있다. Reward signal을 통해서는 오로지 agent가 최종적으로 성취하고자 하는 것에 대해서만 명시하는 것이 좋다.

### 3.3 Returns and Episodes

3.2에서 agent의 목표는 장기간 reward의 누적합에 대한 기대값을 최대화시키는 것이라 하였다. 여기서 '장기간 reward의 누적합'을 *return*이라 정의하면 agent의 목표는 곧 *expected return*을 구하는 것이라 할 수 있다. 가장 간단한 return은 rewards에 대한 단순 합이다.

$$
G_t \doteq R_{t+1} + R_{t+2} + R_{t+3} + \cdots + R_T, \\
\text{where } \: T \: \text{ is a final time step.}
$$

위의 return 계산식에서는 $T$라는 final time step이 존재한다고 가정한다. 이처럼 agent와 환경간의 상호작용을 자연스럽게 subsequence들로 나눌 수 있는 경우, 각 subsequence를 *episode*라고 부르고, 독립적인 episode로 나누어질 수 있는 있는 task를 일컬어 *episodic task*라고 한다. Episodic task에서는 episode의 끝을 알리는 특수한 state인 *terminal state*가 존재한다. 상황에 따라 terminal state를 제외한 모든 states (nonterminal states)를 언급해야 할 때가 있는데, 이는 $\mathcal{S}$로 표기하고 terminal state를 포함한 모든 states는 $\mathcal{S}^+$로 표기한다.

반면 agent와 환경의 상호작용을 어떤 subsequence로 쪼갤 수 없는 경우, 이를 *continuing task*라고 부른다 (가령 24시간 무중단 시스템). 이때 final step은 $T=\infty$일 것이고 위의 단순합으로 정의된 return은 발산하게 될 것이다. 여기서 continuing task에서도 사용가능한 return을 정의하기 위해 *discounting*이라는 개념을 도입하도록 한다. *Discounted return*은 아래와 같이 정의된다.
$$
G_t \doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1},\\
\text{where } \: \gamma \: \text{ is a parameter, } \: 0 \le \gamma \le 1, \: \text{ called the discount rate.}
$$

$\gamma < 1$일 때는 discounted return이 어떤 상수값으로 수렴하게 될 것이다. 그리고 $\gamma = 0$일 때는 agent가 즉각적인 보상을 최대화하는 방향으로 학습하게 될 것이다. 

$\gamma$의 값이 1에 가까워질수록 return은 미래의 보상을 더욱 강하게 반영한다.

### 3.4 Unified Notation for Episodic and Continuing Tasks

Return은 episodic task와 continuing task에 모두 적용될 수 있도록 아래와 같이 일반화될 수 있다. 

$$
G_t \doteq \sum_{k=t+1}^{T} \gamma^{k-t-1} R_k, \\
\text{including the possibility that } \: T = \infty \: \text{ or } \: \gamma = 1 \: \text{ (but not both)}.
$$

* $T \neq \infty$, $\gamma = 1$: 처음에 소개된 return(rewards의 단순합)과 같아진다.
* $T = \infty$, $0 \le \gamma \le 1$: Discounted return과 동일하다.

### 3.5 Policies and Value Functions

거의 모든 강화학습의 알고리즘에서는 state에 대한 또는 state-action pair에 대한 value function을 추정한다. 이 추정치는 주어진 state 또는 state-action이 agent에게 있어 *얼마나 좋은지(how good)*를 나타내는 수치이다. 여기서 "얼마나 좋은지(how good)"란, 미래에 받을 것이라 여겨지는 보상(더 정확하게는 expected return)에 대한 것이라고 볼 수 있다. 만약 agent의 행동방식을 알고 있다면 잠정적인 보상을 예상할(expect) 수 있기에, value function은 policy(정책)라 불리는, agent가 행동하는 방식에 의해 정의된다.

*Policy*는 어떤 state에서 agent가 각 action을 선택할 확률을 나타낸다. 만약 agent가 policy $\pi$를 따른다면, $S_t = s$ (at time $t$)에서 $A_t = a$를 선택할 확률은 다음과 같다: $\pi(a | s)$. 이는 각 $s \in \mathcal{S}$에 대한 $a \in \mathcal{A}$의 확률분포로 볼 수 있다. 강화학습의 방법론들은 agent의 경험에 따라 policy가 어떻게 변경되는지에 대해 명시한다.

Policy $\pi$를 따르는 state $s$의 state-value function은 $v_{\pi}(s)$로 쓰고, 다음과 같이 MDP에 대한 expected return의 계산식으로 정의한다.
$$
v_{\pi}(s) \doteq \mathbb{E}_{\pi} [ G_t \: | \: S_t = s ] = \mathbb{E}_{\pi} \big[ \sum_{k=t+1}^{T} \gamma^{k-t-1} R_{k} \: | \: S_t = s \big], \: \text{ for all } \: s \in \mathcal{S},\\
\text{ where } \: \mathbb{E}_{\pi}[\cdot] \: \text{ denotes the expected value of a random variable given that the agent follows policy } \: \pi,\\
\text{ and } \: t \text{ is any time step}.
$$

마찬가지로 $\pi$를 따르는 state $s$에서의 action $a$에 대한 action-value function은 다음과 같이 정의한다.
$$
q_{\pi}(s, a) \doteq \mathbb{E}_{\pi} [ G_t \: | \: S_t = s, A_t = a ] = \mathbb{E}_{\pi} \big[ \sum_{k=t+1}^{T} \gamma^{k-t-1} R_{k} \: | \: S_t = s, A_t = a \big].
$$

Reinforcement Learning과 Dynamic programming에서는 전반적으로 위 state-value function을 연이어 등장하는 state-value function들 간의 재귀적인 관계로 전개하여 사용하는데, 이를 $v_{\pi}$에 대한 *Bellman equation*이라 부른다. 
$$
\begin{align*}
v_{\pi}(s) &\doteq \mathbb{E}_{\pi} [ G_t \: | \: S_t = s ]\\
&= \mathbb{E}_{\pi} [R_{t+1} + \gamma G_{t+1} \: | \: S_t = s ]\\
&= \sum_{a} \pi(a \: | \: s) \sum_{s'} \sum_{r} p(s', r \: | \: s, a) \big[ r + \gamma \mathbb{E}_{\pi} [ G_{t+1} \: | \: S_{t+1} = s' ] \big]\\
&= \sum_{a} \pi(a \: | \: s) \sum_{s'} \sum_{r} p(s', r \: | \: s, a) \big[ r + \gamma v_{\pi} (s') \big], \: \text{for all } \: s \in \mathcal{S},\\
\end{align*}\\
$$
$$
\text{where it is implicit that the action, } \: a \text{, are taken from the set } \: \mathcal{A}(s), \: \text{that the next states, } \: s',\\
\text{are taken from the set } \: \mathcal{S} \: \text{( or from } \: \mathcal{S}^+ \: \text{ in the case of an episodic problem),}\\
\text{and that the rewards, } \: r, \text{ are taken from the set } \: \mathcal{R}.
$$

$v_{\pi}(s)$에 대한 backup diagram을 그려보면 아래와 같다. 이는 state $s$에서 도달 가능한 모든 next state들에 대한 value의 가중합산과 같다.
<img src="images/backup_diagram.png" alt="Backup diagram for state value function" style="width: 300px;"/>

**Example 3.5: Gridworld**

Figure3.2(좌측)은 어떤 간단한 finite MDP의 예시를 rectangular gridworld로 표현한 형태이다. 모든 cell에서 동, 서, 남, 북으로 이동하는 4가지 action을 취할 수 있으며, action에 의해 grid 바깥으로 나가게 될 경우 이동하지 않고 -1의 보상을 발생시킨다. 그리고 이외에 나머지 경우에 대해서는 0의 보상이 주어진다. 예외적으로 A와 B 셀에서는 action에 상관없이 정해진 cell로 이동시키며, agent는 각각에 대해 +10과 +5의 보상을 얻게된다.

Agent가 각 방향에 대해 동일한 확률로 이동한다고 가정해보자($\pi$). 이때 $\gamma$가 0.9라면 각 state에 대한 value는 Figure3.3의 우측과 같은 값을 갖는다. 그림을 잘 관찰해보면 negative reward를 받을 가능성이 있는 셀은 상대적으로 낮은 value를, 특수한 셀인 A와 B 그리고 그 근방에 있는 셀들은 상대적으로 높은 value를 갖고 있음을 알 수 있다.
<img src="images/figure3.2.png" alt="gridworld"/>

**Exercise 3.14**

모든 state의 value function $v_{\pi}$에 대해 Bellman equation은 항상 성립해야 한다. Example 3.5의 가운데 셀에 대해 Bellman equation이 성립함을 보여보자. (소수점 첫째 자리까지 표시)

*풀이:*

$$
\begin{align*}
&0.25 \cdot 1 \cdot [0 + 1 \cdot 2.3] +\\
&0.25 \cdot 1 \cdot [0 + 1 \cdot 0.4] +\\
&0.25 \cdot 1 \cdot [0 + 1 \cdot -0.4] +\\
&0.25 \cdot 1 \cdot [0 + 1 \cdot 0.7] &= 0.75\\
&&\approx 0.7
\end{align*}
$$

**Exercise 3.19**

State-action에 대한 backup diagram을 그려본다면 다음 그림과 같을 것이다. 주어진 state $s$와 action $a$에 대한 next state $s'$와 reward $r$의 확률분포를 $p(s', r \: | \: s, a)$라고 할때, $q_{\pi}(s, a)$에 대한 Bellman equation을 유도해보자.
<img src="images/exercise3.19.png" alt="backup diagram for a state-action pair"/>

*풀이:*

\begin{align*}
q_{\pi}(s, a) &\doteq \mathbb{E}_{\pi} [ G_t \: | \: S_t = s, A_t = a ]\\
&= \mathbb{E}_{\pi} [R_{t+1} + \gamma G_{t+1} \: | \: S_t = s, A_t = a ]\\
&= \sum_{s'} \sum_{r} p(s', r \: | \: s, a) \big[ r + \gamma \mathbb{E}_{\pi} [ G_{t+1} \: | \: S_{t+1} = s' ] \big]\\
&= \sum_{s'} \sum_{r} p(s', r \: | \: s, a) \big[ r + \gamma v_{\pi} (s') \big], \: \text{for all } \: s \in \mathcal{S}, \: a \in \mathcal{A}(s).\\
\end{align*}

### 3.6 Optimal Policies and Optimal Value Functions

Reinforcement learning task를 푼다는 것은 장기적으로 많은 보상을 획득할 수 있는 policy를 찾는 것으로 봐도 무방하다. Finite MDP에 대해 optimal policy는 다음과 같은 방법으로 정의할 수 있다. Policy $\pi$에 의한 모든 state에서의 expected return이 policy $\pi'$에 의한 것보다 크거나 같을때, policy $\pi$를 policy $\pi'$보다 더 좋거나 같은 것으로 정의한다. 이는 또한 다음과 같이 표현할 수 있다: $\pi \ge \pi'$ if and only if $v_{\pi}(s) \ge v_{\pi'}(s)$ for all $s \in \mathcal{S}$. 이때 다른 모든 policy들보다 더 좋거나 같은 policy가 항상 적어도 하나 이상 존재하는데, 이를 *optimal policy*라 하고 $\pi_*$로 표현한다. 또한 모든 optimal policy들은 *optimal state-value function*이라 불리는 같은 state-value function을 공유하는데, 이를 다음과 같이 정의한다.

$$
v_*(s) \doteq \max_{\pi} v_{\pi}(s), \text{ for all } \: s \in \mathcal{S}.
$$

Optimal policy들은 또한 같은 *optimal action-value function*을 공유하고, 이는 다음과 같이 정의한다.

$$
q_*(s, a) \doteq \max_{\pi} q_{\pi}(s,a), \text{ for all } \: s \in \mathcal{S} \: \text{ and } \: a \in \mathcal{A}(s).
$$

Optimal value function $v_*$는 특정 policy의 참조 없이도 특별한 형태로써 Bellman equation에 의한 self-consistency condition을 만족한다. 이때의 Bellman equation을 $v_*$에 대한 Bellman equation 또는 *Bellman optimality equation*이라 한다. Bellman optimality equation은 optimal policy를 따르는 어떤 state에 대한 value가 그 state에서 최고 가치의 action을 선택한 경우에 대한 expected return과 같아야 함을 의미한다.

$$
\begin{align*}
v_*(s) &= \max_{a \in \mathcal{A}(s)} q_{\pi_*}(s, a)\\
&= \max_a \mathbb{E}_{\pi_*} [G_t \: | \: S_t = s, A_t = a]\\
&= \max_a \mathbb{E}_{\pi_*} [R_{t+1} + \gamma G_{t+1} \: | \: S_t = s, A_t = a]\\
&= \max_a \mathbb{E} [R_{t+1} + \gamma v_*(S_{t+1}) \: | \: S_t = s, A_t = a]\\
&= \max_a \sum_{s', r} p(s', r \: | \: s, a) [r + \gamma v_*(s') ].
\end{align*}
$$

$q_*$에 대한 Bellman optimality equation은 다음과 같다.

$$
\begin{align*}
q_*(s,a) &= \mathbb{E} \big[ R_{t+1} + \gamma \max_{a'} q_*(S_{t+1}, a') \: | \: S_t = s, A_t = a \big] \\
&= \sum_{s', r} p(s', r \: | \: s, a) \big[ r + \gamma \max_{a'} q_*(s', a') \big].
\end{align*}
$$

아래 figure 3.4는 각각 $v_*$와 $q_*$에 대한 Bellman optimality equation을 backup diagram으로 나타낸 것이다.
<img src="images/figure3.4.png" alt="backup diagram for Bellman optimality equation"/>

Finite MDP에서 Bellman optimality equation은 unique solution을 가진다. Bellman optimality equation은 연립방정식이라 볼 수 있으며, $n$개의 state가 있는 경우 이는 $n$개의 변수를 가진 $n$개의 연립방정식과 같다. 만약 환경을 어떤 확률함수 $p$로 표현할 수 있으며 $p$를 알고있을때, 비선형 연립방정식을 풀 수 있는 임의의 방법을 이용하여 이 연립방정식을 푸는 것이 가능하다.

$v_*$를 알고있다면 optimal policy를 결정하는 것은 상대적으로 쉬워진다. 어떤 state에서 Bellman optimality equation을 최대화시키는 action이 하나 혹은 그 이상 존재할 것인데, optimal policy는 이런 action들에 대해서만 non-zero probability를 할당한다. 다르게 말하면 optimal policy란 optimal evaluation function $v_*$에 대해 greedy한 선택을 하는 policy라 할 수 있다. Greedy selection을 한다고 하면 단기적인 보상만을 염두하는 것처럼도 보일 수 있지만, $v_*$가 이미 장기적인 누적보상을 내포하고 있기 때문에 greedy selection만으로도 long-term optimal action을 고를 수 있게된다.

**Example 3.8: Solving the Gridworld**

Example 3.5의 문제에서 $v_*$에 대한 Bellman equation을 푼다고 가정해보자. Figure 3.5의 중앙에는 optimal value function이, 우측에는 이에 상응하는 optimal policy들이 보여지고 있다. 여러 방향의 화살표가 하나의 셀에 들어있는 경우가 눈에 띄는데, 여기서 모든 화살표들은 optimal actions에 해당한다.

<img src="images/figure3.5.png" alt="Optimal solutions to the gridworld example."/>

### 3.7 Optimality and Approximation

불행하게도 설령 환경에 대한 정확한 모델($p$)이 있더라도 보통은 극도로 높은 계산비용때문에 Bellman optimality equation을 풀어서 optimal policy를 얻는 것은 불가능하다. 더불어 계산비용 뿐만 아니라 메모리 또한 큰 제약사항이 된다. 물론 각 state 또는 state-action에 대한 내용을 array나 table로 표현하여 문제를 해결할 수 있는 경우(*tabular case*)도 존재할 것이다. 하지만 대부분의 경우에는 table로 문제를 표현하는 것에 한계가 있기에, value function, policy, model 같은 것들을 좀 더 작은(compact) 형태의 parameterized function representation을 이용하여 근사해야 한다.

단, approximate optimal policies는 자주 접하지 않는 state보다 자주 접하는 state에 대해 더 큰 공을 들이는 경향이 있고, 이 때문에 자주 마주치는 state들에 대해서 좀 더 좋은 결정을 내리는 경향이 있다. 바로 이것이 MDP를 푸는 다른 근사방식들(other approaches to approximately solving MDPs)과는 다른 reinforcement learning의 on-line nature이다.

### References

##### 1. Sutton, R. and Barto, A. (2017). *Reinforcement Learning: An Introduction*. 2nd ed (Complete Draft). MIT Press
##### 2. Sutton, R. and Barto, A. (2018). *Reinforcement Learning: An Introduction*. 2nd ed. MIT Press